## 1. Introduction
Gradient Boosting là thuật toán học có giám sát thuộc lớp ensemble. Nó kết hợp nhiều model yếu hơn để tạo 1 model mạnh. Thuật toán đệ quy bổ sung thêm các cây vào model với target là pseudo residual và điều chỉnh weight theo gradient của lost function. Mô hình cuối cùng là một weighted sum của các cây trong mô hình. Gradient Boosting là phương pháp rất hiệu quả trong các bàn toán thực tế. 
## 2. Math Foundation

Ta có mô hình tổng quát là weighted sum của các mô hình yếu hơn từ lớp H
$$\hat{F} = \sum_{m=1}^{M}\gamma_m h_m(x) + const$$
Ta approximate $\hat{F}$ bằng cách tìm xấp xỉ các parameters model cho hàm loss function đạt cực tiểu. Ta xấp xỉ theo phương pháp tham lam, tức là ta bắt đầu từ tìm cực tiểu hàm mất mát cho mô hình base, tiếp tục mở rộng làm loss và làm tương tự.
$$F_0(x) = \argmin_{h_m \in H} \sum_{i=1}^nL(y_i, h_m(x_i))$$
$$F_m(x) = F_{m-1}x + \argmin_{h_m \in H} \sum_{i=1}^nL(y_i, F_{m-1}x_i + h_m(x_i))$$
Rất khó để giải nghiệm đúng của phương trình này, ta sẽ dùng phương pháp gradient descent để xấp xi nghiệm
$$F_m(x) = F_{m-1}x-\gamma \sum_{i=1}^n \nabla F_{m-1}L(y_i, F_{m-1}x_i)$$
với $\gamma > 0$ 


**Optimal value for $\gamma$** 
$$\gamma_m = \argmin_\gamma \sum_{i=1}^nL(y_i, F_mx_i)= \argmin_\gamma L(y_i, F_{m-1} - \gamma \nabla F_{m-1}L(y_i, F_{m-1}x_i))$$

## 3. Pseudo code
1. Initialize model with constant value: $F_0x = \argmin_\gamma \sum_{i=1}^nL(y_i, \gamma)$
2. For m = 1 to M:
- Compute pseudo-residuals: 
$$r_im = - \left[ \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right]_{F(x_i) = F_{m-1}(x_i)} \quad \text{for } i = 1, \dots, n.
$$
- Fit a base learner under scaling $h_m(x)$ to pseudo residual
- Compute $\gamma_m$ by solving the optimization problem
$$\gamma_m = \argmin_\gamma \sum_{i-1}^nL(y_i, F_{m-1}x_i+\gamma h_m(x_i))$$
- Update model 
$$F_m(x) = F_{m-1}(x) + \gamma_m h_m(x)$$

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

## 4. Implementation
Tai mỗi node, ta cần tìm một giá trị constant gamma để gán cho giá trị đầu ra của node
Giá trị gamma này cần phải tối ưu, theo nghĩa làm giảm error nhiều nhất có thể.
$$L = \sum(y_i - f(x_i))^2 = \sum(y_i = f_current(x_i) + \gamma) $$
Lấy gradient và giải cho ptr gradient = 0 ta có
$$\frac{\partial L}{\partial y} = -2 \sum(y_i -f_{current}(x_i) - \gamma) = 0 \iff \sum(y_i - f_{current}(x_i)) = \sum(\gamma) $$
$$ \iff \gamma = \frac{y_i-f_{current}x_i}{n}$$


In [96]:
class GradientBoosting():
    def __init__(self, n_estimators = 100, lamda = 1.0):
        """
        parameters: 
        n_estimators (int): number of base model, default 100
        lamda (float): learning rate, default 1.0 
        """    
        self.n_estimators = n_estimators
        self.lamda = lamda
        self.trees = []
    def __initialise(self, y):
        n_observations = len(y)
        self.init_gamma = np.mean(y)
        f0 = np.full((n_observations, ), self.init_gamma)
        return f0
    def _compute_gamma(self, residuals, leaf_ids):
        """
        Compute optimal gamma
        Parameters:
        residuals: error
        leaf_ids: indices of leaves corresponding observation
        return:
        gamma: the optimal value for gamma
        """
        leaves = np.unique(leaf_ids)
        gamma = np.zeros(residuals.shape)
        for leaf in leaves: 
            ids = np.where(leaf_ids == leaf)[0]
            gamma[ids] = np.mean(residuals[ids])
        return gamma
    def fit(self, X, y):
        #initialise base model
        self.f0 = self.__initialise(y)
        #assign current f by f0
        f_current = self.f0
        #loop M timesL
        for i in range(self.n_estimators):
            #calculate residual
            residual = y - f_current
            #fit a tree to residual
            tree = DecisionTreeRegressor()
            tree.fit(X, residual)
            leaf_ids = tree.apply(X)
            gamma = self._compute_gamma(residual, leaf_ids)
            f_current = f_current + self.lamda*gamma
            self.trees.append(tree)
    def predict(self, X):
        size = X.shape[0]
        pred = np.full((size, ), self.init_gamma)
        for tree in self.trees:
            residual = tree.predict(X)
            leaf_ids = tree.apply(X)
            gamma = self._compute_gamma(residual, leaf_ids)
            pred = pred + self.lamda*gamma
        return pred


In [89]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [97]:
df = datasets.load_diabetes()
X, y = df.data, df.target
model = GradientBoosting(n_estimators=100, lamda=0.1)
model.fit(X, y)
y_pred = model.predict(X)

In [98]:
mean_squared_error(y, y_pred)

4.183580705289566e-06

## 5. Thư viện sklearn

In [102]:
from sklearn.ensemble import GradientBoostingRegressor
model_2 = GradientBoostingRegressor(max_depth=6, n_estimators=100, learning_rate=0.1)
model_2.fit(X, y)
y_pred_2 = model_2.predict(X)
mean_squared_error(y_pred_2, y)

28.80808773702266

## 6. Pros and cons 
**Pros**
- Flexibility: có thể sử dụng cho nhiều bài toán: phân loại, regression, ranking
- Can handle non linear relationship: Gradient boosting có thể thể hiện mối quan hệ phức tạp giữa biến đầu vào và biến đầu ra bằng cách kết hợp nhiều decision tree với weight khác nhau
- High accuracy: được đánh giá là có độ chính xác cao và thường được sử dụng trogn nhiều cuộc thi
- Feature selection: naturally identifies and prioritise the most important features
- Reduction to bias: by sequential error correction and region - specific adjustment. Areas with high bias receive more attention in subsequent iterations. Each tree divides the feature space into regions (leaves) and provides specific corrections for each region
This allows the model to learn different adjustments for different parts of the feature space.
- Robustness to missing data
- Relatively few assumption: does not require feature scaleing, no assumption about distribution of features or target
-Handles unbalanced data: can be wieghted to addess class imbalance issues


**Cons** 
- Overfitting risk
- Computational intensity: can not be parallelised, training can be low
- Memory requirement: entrire ensemple must be stored
- Sensitive to outliers: the algorithm focus on correcting errors, which can cause it to overfit
- Hyperparameter tuning complexity: require carefule tuning of multiple parameters( number of trees, tree depth, learning rate)
- Limited Interpretability: full ensemple becomes a blackbox
- Poor performance with sparse features: may not work for very sparse high dimensional data